In [20]:
# This notebook is used to extract a Job Description from a Linkedin Job's ID

# Import the necessary libraries
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup as bs4
from time import sleep


In [21]:
DRIVER_PATH = '../driver/mac/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.maximize_window()

In [22]:
def login():
      sleep(2)
      url = 'https://www.linkedin.com/login'
      driver.get(url)
      print('- Finish initializing a driver')
      sleep(2)

      # Task 1.2: Import username and password
      credential = open('credentials.txt')
      line = credential.readlines()
      username = line[0]
      password = line[1]
      print('- Finish importing the login credentials')
      sleep(1)

      # Task 1.2: Key in login credentials
      email_field = driver.find_element_by_id('username')
      email_field.send_keys(username)
      print('- Finish keying in email')
      sleep(3)

      password_field = driver.find_element_by_name('session_password')
      password_field.send_keys(password)
      print('- Finish keying in password')
      sleep(3)

      # Task 1.2: Click the Login button
      # signin_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
      # signin_field.click()
      # sleep(3)

      print('- Finish Task 1: Login to Linkedin')

      # Task 1.3: Check if the login is successful
login()

- Finish initializing a driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task 1: Login to Linkedin


In [23]:
def extract_JD(job_id=''):
        # Task 2.1: Access the job page
        url = 'https://www.linkedin.com/jobs/view/' + job_id
        driver.get(url)
        sleep(2.5)
        jobTitle=''
        location =''
        company = ''
        description = ''
        type = ''
        JD = {'id':job_id,'jobTitle':jobTitle, 'location':location, 'company':company, 'type':type, 'description':description}
        try:
                button_showmore = driver.find_elements_by_css_selector("[aria-label='Click to see more description'")
                button_showmore[0].click()
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except:
                print('- Cannot find the button to show more description')
                
                return JD
        # Task 2.2: Extract the Job Description
        jobTitle = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > h1').text
        location = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt2 > span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black > span.jobs-unified-top-card__bullet').text
        company = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt2 > span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black > span.jobs-unified-top-card__company-name').text
        type = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt5.mb2 > ul > li:nth-child(1) > span').text
        # status = driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div:nth-child(1) > div > div.p5 > div.mt5.mb2 > ul > li:nth-child(4) > span').text
        description =  driver.find_element_by_css_selector('body > div.application-outlet > div.authentication-outlet > div > div.job-view-layout.jobs-details > div.grid > div > div.jobs-box--fadein.jobs-box--full-width.jobs-box--with-cta-large.jobs-description.artdeco-card.mb4.jobs-poster--is-expanded-redesign > article').text.strip()
        
        JD = {'id':job_id,'jobTitle':jobTitle, 'location':location, 'company':company, 'type':type, 'description':description}
        return JD

        # Task 2.3: Save the Job Description to a csv file
    

In [7]:
listJD = extract_JD('2949802590')
print(listJD)

{'id': '2949802590', 'jobTitle': 'Data Engineer', 'location': 'Ho Chi Minh City, Vietnam', 'company': 'Senspark', 'type': 'Full-time', 'description': 'About the job\nMô tả công việc:\nXây dựng cơ sở hạn tầng cần thiết để trích xuất, chuyển đổi và tải dữ liệu tối ưu từ nhiều nguồn dữ liệu khác nhau.\nPhát triển hệ thống cơ sở dữ liệu phục vụ cho các hoạt động Game Marketing và Game Product của công ty.\nPhối hợp với bộ phận Data Analyst, Marketing, Game Designer để xây dựng các công cụ hỗ trợ công việc báo cáo.\nTối ưu và đảm bảo tính chính xác, sự ổn định và bảo mật của dữ liệu.\nLuôn cập nhật tình hình các thư viện hỗ trợ lấy dữ liệu (Firebase, Google Analytics, 3rd Party Ads API…).\nCác công việc khác theo yêu cầu của cấp trên.\nYêu cầu công việc:\nÍt nhất 3 năm kinh nghiệm về Data Engineer (ưu tiên có kinh nghiệm về mảng Game Mobile và Blockchain).\nTốt nghiệp đại học chuyên ngành về Khoa học máy tính, Thống kê, Tin học, Hệ thống thông tin… (ưu tiên tốt nghiệp các trường đại học Quố

In [25]:
import glob
def crawl(folder_id='',output_folder=''):
    mylist = [f for f in glob.glob(folder_id + "/Database Administrator.csv")]
    fields = ['id','jobTitle','location', 'company', 'type', 'description']
    for i in mylist:
        print('- Start crawling job description from ' + i)
        result_list = []
        list_id = pd.read_csv(i,header=None)[1].values.tolist()
        for j in list_id:
            jd = extract_JD(str(j))
            print(jd)
            result_list.append(jd)
            # print(jd)
        print("Finish crawling the file: " + i)
        df = pd.DataFrame(result_list)
        df.to_csv(output_folder + '/JD_' + i.split('/')[-1][0:-4] +'.csv',index=False,header=False,sep='|',mode='a')
crawl('jobs_result','output')

- Start crawling job description from jobs_result/Database Administrator.csv
{'id': '2987768901', 'jobTitle': 'Database Administrator (Chuyên Viên Quản Trị Cơ Sở Dữ Liệu)', 'location': 'Hanoi, Hanoi, Vietnam', 'company': 'Techcombank (TCB)', 'type': 'Full-time · Entry level', 'description': 'About the job\nMục tiêu\nCung cấp dịch vụ Cơ sở dữ liệu và phần mềm nền cho các hệ thống\nĐảm bảo các hệ thống cơ sở dữ liệu và phần mềm nền hoạt động liên tục, ổn định\nĐảm bảo các hệ thống cơ sở dữ liệu và phần mềm nền có khả năng mở rộng linh hoạt, đáp ứng nhu cầu tăng trưởng nhanh của ngân hàng\nĐảm bảo các hệ thống cơ sở dữ liệu và phần mềm nền đáp ứng những tiêu chuẩn công nghệ và tiêu chuẩn an ninh thông tin của Techcombank\nMô tả công việc\nGiám sát và khắc phục sự cố của Cơ sở dữ liệu, SQL\nPhát triển, vận hành và quản lý Cơ sở dữ liệu\nThiết lập, phân quyền người dùng thao tác với Cơ sở dữ liệu\nQuản lý chính sách, qui trình Cơ sở dữ liệu trình để đảm bảo an ninh\nQuản lý sao lưu / khôi p

In [12]:
from ast import literal_eval
l = []
with open("temp.txt", "r") as ins:
    for line in ins:
        your_dict = literal_eval(line)
        l.append(your_dict)
df = pd.DataFrame(l)
df.to_csv('output' + '/JD_Software Engineer' +'.csv',index=False,header=True,sep='|')
        # print(your_dict["jobTitle"])
# df = pd.read_csv('output/JD_Back End Developer.csv',sep='|')
# print(df)